In [1]:
# Ariel Data Challenge 2024: Exoplanet Spectrum Analysis Pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter1d
from scipy.optimize import minimize
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import gc
import logging
from tqdm.notebook import tqdm
import os

In [2]:
# 1. Data Loading Functions

def load_planet_data(planet_id, data_path, is_test=False):
    folder = "test" if is_test else "train"
    airs_signal_path = f"{data_path}/{folder}/{planet_id}/AIRS-CH0_signal.parquet"
    
    if not os.path.exists(airs_signal_path):
        print(f"File not found: {airs_signal_path}")
        return None, None, None
    
    try:
        airs_signal = pd.read_parquet(airs_signal_path)
        fgs1_signal = pd.read_parquet(f"{data_path}/{folder}/{planet_id}/FGS1_signal.parquet")
        
        calibration_files = {
            'dark': pd.read_parquet(f"{data_path}/{folder}/{planet_id}/AIRS-CH0_calibration/dark.parquet"),
            'flat': pd.read_parquet(f"{data_path}/{folder}/{planet_id}/AIRS-CH0_calibration/flat.parquet"),
            'dead': pd.read_parquet(f"{data_path}/{folder}/{planet_id}/AIRS-CH0_calibration/dead.parquet"),
            'linear_corr': pd.read_parquet(f"{data_path}/{folder}/{planet_id}/AIRS-CH0_calibration/linear_corr.parquet")
        }
        
        return airs_signal, fgs1_signal, calibration_files
    except Exception as e:
        print(f"Error loading data for planet {planet_id}: {str(e)}")
        return None, None, None

In [3]:
# 2. Data Preprocessing Functions

def reshape_and_calibrate(airs_signal, calibration_files):
    airs_data = airs_signal.values.reshape(-1, 32, 356)
    airs_data = (airs_data - calibration_files['dark'].values) / calibration_files['flat'].values
    return airs_data

def extract_spectral_data(airs_data):
    return np.mean(airs_data, axis=1)

def measure_centroid(image):
    y, x = np.indices(image.shape)
    total = image.sum()
    x_center = (x * image).sum() / total
    y_center = (y * image).sum() / total
    return x_center, y_center

def correct_jitter(airs_data, fgs1_data, airs_time, fgs1_time):
    # Ensure FGS1 data covers the full AIRS-CH0 time range
    fgs1_start = max(fgs1_time.min(), airs_time.min())
    fgs1_end = min(fgs1_time.max(), airs_time.max())
    mask = (fgs1_time >= fgs1_start) & (fgs1_time <= fgs1_end)
    fgs1_time = fgs1_time[mask]
    fgs1_data = fgs1_data[mask]

    # Interpolate FGS1 centroid positions to AIRS-CH0 timestamps
    interp_x = interp1d(fgs1_time, fgs1_data[:, 0], kind='cubic', fill_value='extrapolate')
    interp_y = interp1d(fgs1_time, fgs1_data[:, 1], kind='cubic', fill_value='extrapolate')
    
    x_pos = interp_x(airs_time)
    y_pos = interp_y(airs_time)

    # Smooth the centroid positions to reduce noise
    x_smooth = savgol_filter(x_pos, window_length=51, polyorder=3)
    y_smooth = savgol_filter(y_pos, window_length=51, polyorder=3)

    # Calculate pixel shifts
    x_shift = x_smooth - np.median(x_smooth)
    y_shift = y_smooth - np.median(y_smooth)

    # Correct AIRS-CH0 data for jitter
    corrected_data = np.zeros_like(airs_data)
    for i in range(airs_data.shape[1]):
        # Create a 2D interpolation function for each wavelength
        interp_func = interp1d(airs_time, airs_data[:, i], kind='cubic', fill_value='extrapolate')
        
        # Apply correction
        corrected_time = airs_time - x_shift * 0.1 - y_shift * 0.1  # Adjust scaling factors as needed
        corrected_data[:, i] = interp_func(corrected_time)

    return corrected_data

def normalize_data(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    return (data - mean) / (std + 1e-10)

In [4]:
# 3. Model Building Functions

def simple_transit_model(params, time):
    t0, per, depth, duration = params
    phase = (time - t0 + 0.5*per) % per - 0.5*per
    transit = np.abs(phase) < 0.5*duration
    return 1 - depth * transit

def fit_transit_model(flux, time):
    def residuals(params):
        return np.sum((flux - simple_transit_model(params, time))**2)
    
    initial_guess = [np.median(time), 0.1 * (time[-1] - time[0]), 0.01, 0.1 * (time[-1] - time[0])]
    bounds = ((0, len(time)), (0, len(time)), (0, 0.1), (0, len(time)/2))
    result = minimize(residuals, initial_guess, bounds=bounds, method='L-BFGS-B')
    return result.x

def process_planet(planet_id, data_path, axis_info, is_test=False):
    print(f"Processing {'test' if is_test else 'train'} planet {planet_id}")
    # Load data
    airs_signal, fgs1_signal, calibration_files = load_planet_data(planet_id, data_path, is_test)
    
    if airs_signal is None or fgs1_signal is None or calibration_files is None:
        print(f"Skipping planet {planet_id} due to data loading error")
        return None

    # Preprocess data
    airs_data = reshape_and_calibrate(airs_signal, calibration_files)
    spectral_data = extract_spectral_data(airs_data)
    
    # Time arrays
    airs_time_step = axis_info['AIRS-CH0-axis0-h'].iloc[1] - axis_info['AIRS-CH0-axis0-h'].iloc[0]
    time = np.arange(len(spectral_data)) * airs_time_step
    
    fgs1_data = fgs1_signal.values.reshape(-1, 32, 32)
    fgs1_centroids = np.array([measure_centroid(frame) for frame in fgs1_data])
    fgs1_time_step = axis_info['FGS1-axis0-h'].iloc[1] - axis_info['FGS1-axis0-h'].iloc[0]
    fgs1_time = np.arange(len(fgs1_data)) * fgs1_time_step
    
    # Jitter correction
    corrected_spectral_data = correct_jitter(spectral_data, fgs1_centroids, time, fgs1_time)
    
    # Fit transit model for each wavelength
    transit_params = np.array([fit_transit_model(corrected_spectral_data[:, i], time) 
                               for i in range(corrected_spectral_data.shape[1])])
    
    # Extract spectrum and estimate uncertainties
    spectrum = np.mean(corrected_spectral_data, axis=0)
    uncertainties = np.std(corrected_spectral_data, axis=0) / np.sqrt(len(corrected_spectral_data))
    
    result = {
        'wavelengths': pd.read_csv(f"{data_path}/wavelengths.csv").iloc[0].values,
        'spectrum': spectrum,
        'uncertainties': uncertainties,
        'transit_params': transit_params
    }
    
    print(f"Processed planet {planet_id}:")
    print(f"Spectrum shape: {result['spectrum'].shape}")
    print(f"Uncertainties shape: {result['uncertainties'].shape}")
    print(f"Transit params shape: {result['transit_params'].shape}")
    
    return result

In [5]:
# 4. Main Processing Loop
def process_all_planets(data_path, adc_info, axis_info, is_test=False):
    all_results = {}
    total_planets = len(adc_info.index)
    
    for i, planet_id in enumerate(tqdm(adc_info.index, desc="Processing planets")):
        results = process_planet(str(planet_id), data_path, axis_info, is_test)
        if results is not None:
            all_results[planet_id] = results
        
        # Only print progress for datasets with more than 10 planets
        if total_planets > 10:
            if (i + 1) % (total_planets // 10) == 0:
                print(f"Processed {i + 1}/{total_planets} planets")
        else:
            print(f"Processed planet {i + 1}/{total_planets}")
    
    return all_results

In [6]:
# 5. Model Evaluation and Analysis

def load_processed_results(results_path, planet_ids):
    all_results = {}
    for planet_id in planet_ids:
        all_results[planet_id] = np.load(f"{results_path}/planet_{planet_id}_results.npz", allow_pickle=True)
    return all_results

def prepare_data_for_modeling(all_results):
    X = np.array([results['spectrum'] for results in all_results.values()])
    y = np.array([results['transit_params'][:, 2] for results in all_results.values()])  # Using transit depth as target
    return X, y

def train_and_evaluate_models(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # PCA
    pca = PCA(n_components=0.95)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # Linear Regression
    lr = LinearRegression()
    lr.fit(X_train_pca, y_train)
    lr_score = lr.score(X_test_pca, y_test)
    
    # Random Forest
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X_train_pca, y_train)
    rf_score = rf.score(X_test_pca, y_test)
    
    print(f"Linear Regression R2 Score: {lr_score:.4f}")
    print(f"Random Forest R2 Score: {rf_score:.4f}")
    
    return lr, rf, pca, X_test_pca, y_test

In [7]:
# 5.2. Final Model Selection

def select_best_model(lr_model, rf_model, X_test_pca, y_test):
    lr_mse = mean_squared_error(y_test, lr_model.predict(X_test_pca))
    rf_mse = mean_squared_error(y_test, rf_model.predict(X_test_pca))
    
    print(f"Linear Regression MSE: {lr_mse:.6f}")
    print(f"Random Forest MSE: {rf_mse:.6f}")
    
    if lr_mse < rf_mse:
        print("Linear Regression model selected.")
        return lr_model
    else:
        print("Random Forest model selected.")
        return rf_model

In [8]:
# 5.4. Uncertainty Estimation

def estimate_uncertainty(model, X_pca, n_bootstrap=100):
    predictions = []
    for _ in range(n_bootstrap):
        if isinstance(model, RandomForestRegressor):
            # Randomly select estimators and average their predictions
            n_estimators = len(model.estimators_)
            selected_estimators = np.random.choice(model.estimators_, size=n_estimators, replace=True)
            preds = np.mean([estimator.predict(X_pca) for estimator in selected_estimators], axis=0)
        elif isinstance(model, LinearRegression):
            # Add noise to the coefficients
            coef_noise = np.random.normal(0, 0.1, size=model.coef_.shape)
            preds = X_pca @ (model.coef_ + coef_noise).T + model.intercept_
        else:
            raise ValueError("Unsupported model type")
        predictions.append(preds)
    
    return np.std(predictions, axis=0).flatten()  # Ensure 1D array output

In [9]:
# 5.6. Submission Preparation
def prepare_submission(model, pca_model, all_results, output_file):
    planet_ids = list(all_results.keys())
    all_data = []
    
    for planet_id in planet_ids:
        spectrum = all_results[planet_id]['spectrum']
        spectrum_pca = pca_model.transform(spectrum.reshape(1, -1))
        predicted_spectrum = model.predict(spectrum_pca).flatten()
        spectrum_uncertainty = estimate_uncertainty(model, spectrum_pca)
        
        # Ensure we have exactly 283 values for both spectrum and uncertainty
        predicted_spectrum = predicted_spectrum[:283]  # Truncate or pad to 283
        spectrum_uncertainty = spectrum_uncertainty[:283]  # Truncate or pad to 283
        
        # Pad with zeros if less than 283
        if len(predicted_spectrum) < 283:
            predicted_spectrum = np.pad(predicted_spectrum, (0, 283 - len(predicted_spectrum)))
        if len(spectrum_uncertainty) < 283:
            spectrum_uncertainty = np.pad(spectrum_uncertainty, (0, 283 - len(spectrum_uncertainty)))
        
        # Combine all data for this planet
        planet_data = [planet_id] + list(predicted_spectrum) + list(spectrum_uncertainty)
        all_data.append(planet_data)
    
    # Create column names
    column_names = ['planet_id'] + [f'spectrum_{i}' for i in range(283)] + [f'uncertainty_{i}' for i in range(283)]
    
    # Create DataFrame
    submission_df = pd.DataFrame(all_data, columns=column_names)
    
    # Convert all columns except 'planet_id' to float, with error handling
    for col in submission_df.columns[1:]:
        try:
            submission_df[col] = submission_df[col].astype(float)
        except ValueError as e:
            print(f"Error converting column {col} to float:")
            print(submission_df[col].head())
            print(f"Error message: {str(e)}")
            raise
    
    # Save to CSV
    submission_df.to_csv(output_file, index=False)
    print(f"Submission file saved to {output_file}")

    # Verify the shape and content
    print(f"Submission file shape: {submission_df.shape}")
    print(f"Number of columns: {len(submission_df.columns)}")
    print("First few columns:", submission_df.columns[:5].tolist())
    print("Last few columns:", submission_df.columns[-5:].tolist())
    print(f"Number of spectrum columns: {sum('spectrum_' in col for col in submission_df.columns)}")
    print(f"Number of uncertainty columns: {sum('uncertainty_' in col for col in submission_df.columns)}")
    
    # Print a sample of the data to verify format
    print("\nSample of the first row:")
    print(submission_df.iloc[0, :5].to_dict())  # First 5 columns
    print("...")
    print(submission_df.iloc[0, -5:].to_dict())  # Last 5 columns

In [10]:
if __name__ == "__main__":
    try:
        data_path = "/kaggle/input/ariel-data-challenge-2024"
        train_adc_info = pd.read_csv(f"{data_path}/train_adc_info.csv", index_col='planet_id')
        test_adc_info = pd.read_csv(f"{data_path}/test_adc_info.csv", index_col='planet_id')
        axis_info = pd.read_parquet(f"{data_path}/axis_info.parquet")
        
        print("Processing training data")
        train_results = process_all_planets(data_path, train_adc_info, axis_info)
        
        print("Preparing data for modeling")
        X, y = prepare_data_for_modeling(train_results)
        print(f"Shape of X: {X.shape}")
        print(f"Shape of y: {y.shape}")

        print("Training and evaluating models")
        lr_model, rf_model, pca_model, X_test_pca, y_test = train_and_evaluate_models(X, y)
        print(f"Number of PCA components: {pca_model.n_components_}")
        print(f"Shape of X_test_pca: {X_test_pca.shape}")

        print("Selecting best model")
        best_model = select_best_model(lr_model, rf_model, X_test_pca, y_test)
        
        print("Processing test data")
        test_results = process_all_planets(data_path, test_adc_info, axis_info, is_test=True)
        
        print("Preparing submission")
        prepare_submission(best_model, pca_model, test_results, "submission.csv")

        print("Analysis completed successfully")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise

Processing training data


Processing planets:   0%|          | 0/673 [00:00<?, ?it/s]

Processing train planet 785834
Processed planet 785834:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing train planet 14485303
Processed planet 14485303:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing train planet 17002355
Processed planet 17002355:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing train planet 24135240
Processed planet 24135240:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing train planet 25070640
Processed planet 25070640:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing train planet 26372015
Processed planet 26372015:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing train planet 29348276
Processed planet 29348276:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processing

Processing planets:   0%|          | 0/1 [00:00<?, ?it/s]

Processing test planet 499191466
Processed planet 499191466:
Spectrum shape: (356,)
Uncertainties shape: (356,)
Transit params shape: (356, 4)
Processed planet 1/1
Preparing submission
Submission file saved to submission.csv
Submission file shape: (1, 567)
Number of columns: 567
First few columns: ['planet_id', 'spectrum_0', 'spectrum_1', 'spectrum_2', 'spectrum_3']
Last few columns: ['uncertainty_278', 'uncertainty_279', 'uncertainty_280', 'uncertainty_281', 'uncertainty_282']
Number of spectrum columns: 283
Number of uncertainty columns: 283

Sample of the first row:
{'planet_id': 499191466.0, 'spectrum_0': 0.0, 'spectrum_1': 0.0, 'spectrum_2': 0.009999999999999998, 'spectrum_3': 0.0}
...
{'uncertainty_278': 0.004221374183841073, 'uncertainty_279': 0.004631940737962854, 'uncertainty_280': 0.004209566604770611, 'uncertainty_281': 0.004488925372513995, 'uncertainty_282': 0.004230153188715505}
Analysis completed successfully
